### Documentación del proyecto final de Aprendizaje Estadístico 2024-2

**Integrantes:** Alejandra Arciniegas Marin - Andrés Felipe Riaño Quintanilla - William Esneider Galeano Sierra

# **Localización**

# Entrenamiento y prueba preliminar del modelo:

```Python

#Librerías:
import os
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

```Python

# Directorio de imágenes y CSVs
image_dir = "./Rets"
fovea_csv_path = "./IDRiD_Fovea_Center.csv"
od_csv_path = "./IDRiD_OD_Center.csv"
output_mask_dir = "./Masks"
os.makedirs(output_mask_dir, exist_ok=True)

```Python

# Parámetros de las máscaras
image_size = (224, 224)  # Tamaño redimensionado de las imágenes
background_gray = 128  # Color del fondo de la máscara
fovea_circle_radius = 10  # Radio del círculo negro para la fóvea
od_circle_radius = 10  # Radio del círculo blanco para el disco óptico

```Python

# Leer CSVs
fovea_data = pd.read_csv(fovea_csv_path)
od_data = pd.read_csv(od_csv_path)

```Python

# Combinar las coordenadas en un DataFrame
coordinates_df = pd.merge(fovea_data, od_data, on="Image No", suffixes=("_fovea", "_od"))

```Python

# Función para crear una máscara de segmentación personalizada
def create_mask(fovea_coords, od_coords):
    """
    Crea una máscara con dos regiones diferenciadas:
    - Un círculo negro para la fóvea
    - Un círculo blanco para el disco óptico

    Parámetros:
    - fovea_coords: coordenadas de la fóvea en el formato (x, y)
    - od_coords: coordenadas del disco óptico en el formato (x, y)
    
    Retorno:
    - mask: imagen en escala de grises con la máscara generada
    """

    # Crear un fondo gris uniforme del tamaño de la imagen
    # image_size[::-1]: dimensiones de la imagen en formato (ancho, alto)
    # dtype=np.uint8: tipo de dato para imágenes de 8 bits (valores entre 0 y 255)
    # background_gray: nivel de gris para el fondo
    mask = np.ones((*image_size[::-1],), dtype=np.uint8) * background_gray

    # Dibujar un círculo negro para la fóvea
    # Las coordenadas se ajustan proporcionalmente al tamaño de la imagen original (4288x2848)
    # fovea_circle_radius: radio del círculo de la fóvea
    # (0): color negro para la región de la fóvea
    # -1: relleno completo del círculo
    cv2.circle(mask, 
                (int(fovea_coords[0] * image_size[0] / 4288), 
                 int(fovea_coords[1] * image_size[1] / 2848)), 
                fovea_circle_radius, (0), -1)

    # Dibujar un círculo blanco para el disco óptico
    # od_circle_radius: radio del círculo del disco óptico
    # (255): color blanco para la región del disco óptico
    cv2.circle(mask, 
                (int(od_coords[0] * image_size[0] / 4288), 
                 int(od_coords[1] * image_size[1] / 2848)), 
                od_circle_radius, (255), -1)

    return mask

```Python

# Generación de máscaras de segmentación
print("Generando máscaras...")

# Iterar sobre cada fila del DataFrame que contiene las coordenadas de la fóvea y el disco óptico
for _, row in tqdm(coordinates_df.iterrows(), total=len(coordinates_df)):
    # Obtener el nombre de la imagen
    image_name = row["Image No"]

    # Obtener las coordenadas de la fóvea y del disco óptico
    fovea_coords = (row["X- Coordinate_fovea"], row["Y - Coordinate_fovea"])
    od_coords = (row["X- Coordinate_od"], row["Y - Coordinate_od"])

    # Crear la máscara utilizando las coordenadas proporcionadas
    mask = create_mask(fovea_coords, od_coords)

    # Definir la ruta de salida para guardar la máscara generada
    output_path = os.path.join(output_mask_dir, f"{image_name}_mask.png")

    # Guardar la máscara como imagen PNG en la ubicación definida
    cv2.imwrite(output_path, mask)

print(f"Máscaras generadas y guardadas en {output_mask_dir}")

```Python

from tensorflow.keras import layers, Model

# Definición de un bloque de atención para la arquitectura Attention U-Net
def attention_block(x, g, inter_channel):
    """
    Bloque de atención que ajusta la importancia de las características de una capa en el decoder.
    
    Parámetros:
    - x: tensor de entrada de la capa a la que se aplica la atención.
    - g: tensor de entrada guía del decoder.
    - inter_channel: número de canales intermedios.

    Retorna:
    - Tensor resultante después de aplicar la atención.
    """
    # Proyección espacial de la entrada y el tensor guía
    theta_x = layers.Conv2D(inter_channel, (1, 1))(x)
    phi_g = layers.Conv2D(inter_channel, (1, 1))(g)

    # Suma de tensores proyectados y activación
    concat = layers.Add()([theta_x, phi_g])
    concat = layers.Activation('relu')(concat)

    # Psi genera una máscara de atención (probabilidades)
    psi = layers.Conv2D(1, (1, 1), activation='sigmoid')(concat)

    # Multiplicación para aplicar la atención
    return layers.Multiply()([x, psi])


# Definición de un bloque convolucional básico
def conv_block(x, num_filters):
    """
    Aplica dos convoluciones con Batch Normalization y ReLU.

    Parámetros:
    - x: tensor de entrada.
    - num_filters: número de filtros para las convoluciones.

    Retorna:
    - Tensor después de aplicar las convoluciones y normalización.
    """
    x = layers.Conv2D(num_filters, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(num_filters, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x


# Definición del modelo Attention U-Net
def attention_unet(input_shape=(224, 224, 3)):
    """
    Crea un modelo Attention U-Net para segmentación de imágenes.

    Parámetro:
    - input_shape: tamaño de entrada de las imágenes.

    Retorna:
    - Modelo de Keras Attention U-Net.
    """
    # Entrada del modelo
    inputs = layers.Input(shape=input_shape)

    # --- Encoder ---
    c1 = conv_block(inputs, 64)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = conv_block(p1, 128)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = conv_block(p2, 256)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = conv_block(p3, 512)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    c5 = conv_block(p4, 1024)

    # --- Decoder ---
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    a6 = attention_block(c4, u6, 512)
    u6 = layers.Concatenate()([u6, a6])
    c6 = conv_block(u6, 512)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    a7 = attention_block(c3, u7, 256)
    u7 = layers.Concatenate()([u7, a7])
    c7 = conv_block(u7, 256)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    a8 = attention_block(c2, u8, 128)
    u8 = layers.Concatenate()([u8, a8])
    c8 = conv_block(u8, 128)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u8)
    a9 = attention_block(c1, u9, 64)
    u9 = layers.Concatenate()([u9, a9])
    c9 = conv_block(u9, 64)

    # Capa de salida: un canal binario para segmentación
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    # Definición del modelo final
    model = Model(inputs, outputs)
    return model

```Python

import tensorflow as tf

# Definición de la métrica Dice Coefficient
def dice_coefficient(y_true, y_pred):
    """
    Calcula el coeficiente de Dice, una métrica de similitud utilizada 
    comúnmente para evaluar la calidad de segmentaciones en imágenes.

    Parámetros:
    - y_true: Tensor de las máscaras reales.
    - y_pred: Tensor de las máscaras predichas.

    Retorna:
    - Valor del coeficiente de Dice, que varía entre 0 y 1:
      * 1 indica una superposición perfecta.
      * 0 indica ninguna superposición.
    """
    # Aplanar los tensores de las máscaras (convertirlos a vectores)
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])

    # Calcular la intersección entre la máscara real y la predicha
    intersection = tf.reduce_sum(y_true_f * y_pred_f)

    # Fórmula del coeficiente de Dice con suavizado para evitar divisiones por cero
    return (2. * intersection + 1) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + 1)

```Python

from functools import partial
import tensorflow as tf

def iou_for_value(y_true, y_pred, value, threshold=0.1):
    """
    Calcula la métrica Intersection over Union (IoU) para un valor específico de interés 
    (por ejemplo, fóvea o disco óptico) en segmentación de imágenes.

    Parámetros:
    - y_true: Tensor de las máscaras reales.
    - y_pred: Tensor de las máscaras predichas.
    - value: Valor de interés en la máscara (0.0 para la fóvea, 1.0 para el disco óptico).
    - threshold: Umbral para considerar un píxel como parte de la región de interés.

    Retorna:
    - IoU: Intersection over Union para la región definida por el valor.
    """
    # Crear máscaras booleanas según el valor objetivo y el umbral
    true_mask = tf.math.abs(y_true - value) < threshold
    pred_mask = tf.math.abs(y_pred - value) < threshold

    # Calcular la intersección y la unión de las máscaras
    intersection = tf.reduce_sum(tf.cast(tf.logical_and(true_mask, pred_mask), tf.float32))
    union = tf.reduce_sum(tf.cast(tf.logical_or(true_mask, pred_mask), tf.float32))

    # Devolver el resultado del IoU, evitando divisiones por cero
    return intersection / (union + tf.keras.backend.epsilon())

# Crear funciones parciales para valores específicos de interés
iou_for_fovea = partial(iou_for_value, value=0.0)  # IoU para la fóvea (valor 0.0)
iou_for_od = partial(iou_for_value, value=1.0)  # IoU para el disco óptico (valor 1.0)

# Ajustar los nombres para que se puedan registrar como métricas en TensorFlow
iou_for_fovea.__name__ = "iou_fovea"
iou_for_od.__name__ = "iou_od"

```Python

import tensorflow.keras.backend as K
from tensorflow import logical_and

def background_distance_metric(y_true, y_pred):
    """
    Métrica personalizada para evaluar la distancia del fondo a un valor gris ideal (0.5).
    Penaliza predicciones que se alejen de este valor, lo que ayuda a mantener el fondo en tonos intermedios
    y mejorar la segmentación de regiones relevantes (disco óptico y fóvea).

    Parámetros:
    - y_true: Tensor con las máscaras reales.
    - y_pred: Tensor con las máscaras predichas.

    Retorna:
    - Promedio de la distancia del fondo al valor gris ideal (0.5).
    """
    # Valor ideal para el fondo (gris medio)
    ideal_background_value = 0.5

    # Máscara para seleccionar solo el fondo (excluir disco óptico y fóvea)
    mask_background = K.cast(logical_and(y_true > 0.3, y_true < 0.7), K.floatx())

    # Calcular la distancia del fondo al gris ideal
    distance = K.abs(y_pred - ideal_background_value) * mask_background

    # Promediar las distancias en la región de fondo seleccionada
    return K.mean(distance)

```Python

# Crear y compilar el modelo
model = attention_unet(input_shape=(224, 224, 3))
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy',dice_coefficient, iou_for_fovea, iou_for_od,background_distance_metric])
model.summary()

```Python

# Cargar imágenes y máscaras
image_filenames = sorted([f for f in os.listdir(image_dir) if f.endswith(".jpg")])
mask_filenames = sorted([f for f in os.listdir(output_mask_dir) if f.endswith(".png")])

X = []
y = []

```Python

print("Cargando imágenes y máscaras...")

# Iterar sobre las listas de archivos de imágenes y máscaras
for img_file, mask_file in tqdm(zip(image_filenames, mask_filenames), total=len(image_filenames)):
    # Construir las rutas completas de los archivos de imagen y máscara
    img_path = os.path.join(image_dir, img_file)
    mask_path = os.path.join(output_mask_dir, mask_file)

    # Leer la imagen desde el archivo
    img = cv2.imread(img_path)
    # Redimensionar la imagen al tamaño especificado para ajustarla a la memoria y al modelo
    img = cv2.resize(img, image_size)

    # Leer la máscara en escala de grises
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    # Redimensionar la máscara al tamaño especificado
    mask = cv2.resize(mask, image_size)
    # Expandir la dimensión de la máscara para convertirla en (altura, ancho, 1)
    # Normalizar los valores de la máscara a [0, 1]
    mask = np.expand_dims(mask, axis=-1) / 255.0

    # Añadir la imagen y la máscara a las listas
    X.append(img)
    y.append(mask)

# Convertir las listas a arreglos de NumPy
X = np.array(X)
y = np.array(y)

```Python

# División en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Callback para mostrar imágenes durante el entrenamiento
class ImageDisplayCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        idx = np.random.randint(0, len(X_val))
        img = X_val[idx]
        true_mask = y_val[idx]
        pred_mask = model.predict(np.expand_dims(img, axis=0))[0]

        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.title("Máscara Real")
        plt.imshow(true_mask[:, :, 0], cmap='gray')

        plt.subplot(1, 2, 2)
        plt.title("Predicción del Modelo")
        plt.imshow(pred_mask[:, :, 0], cmap='gray')

        plt.show()

# Callbacks
checkpoint = ModelCheckpoint("attention_unet_bestv2.h5", save_best_only=True, monitor="val_loss", mode="min")
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

```Python

# Entrenamiento del modelo
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=4,
    callbacks=[checkpoint, early_stopping,ImageDisplayCallback()]
)

print("Entrenamiento finalizado.")

```Python

from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError  # O usa la métrica exacta que usaste

# Cargar el modelo con la métrica personalizada registrada
modelo_cargado = load_model('attention_unet_bestv2.h5')#, custom_objects={'mse': MeanSquaredError()})

# Verificar la arquitectura del modelo cargado
modelo_cargado.summary()

Predict_mask = modelo_cargado.predict(X)

```Python

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc

# Función para graficar curvas de entrenamiento y validación
def plot_training_curves(history):
    """
    Muestra las curvas de métricas de entrenamiento y validación a lo largo de las épocas.

    Parámetros:
    - history: Objeto History de Keras que contiene los valores de las métricas por época.
    """
    metrics_to_plot = ['loss', 'accuracy', 'iou_od', 'iou_fovea', 'background_distance_metric']
    
    # Configurar la figura
    plt.figure(figsize=(15, 8))
    for i, metric in enumerate(metrics_to_plot, 1):
        # Verificar si la métrica existe en el historial
        if metric in history.history:
            plt.subplot(2, 3, i)
            plt.plot(history.history[metric], label=f'Training {metric}')
            plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric}')
            plt.xlabel('Época')
            plt.ylabel(metric)
            plt.title(f'{metric.capitalize()} a lo largo de las épocas')
            plt.legend()
    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(y_true, y_pred):
    """
    Genera y muestra una matriz de confusión basada en valores de colores (negro, gris y blanco).

    Parámetros:
    - y_true: Máscara real (ground truth).
    - y_pred: Máscara predicha por el modelo.
    """
    # Aplanar las matrices de entrada
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()

    # Definir los límites para negro, gris y blanco
    bins = [0, 0.3, 0.7, 1.01]  # Se extiende el último límite ligeramente para incluir el valor 1.0
    # Asignar cada valor a una categoría (0: negro, 1: gris, 2: blanco)
    y_true_binned = np.digitize(y_true_flat, bins) - 1
    y_pred_binned = np.digitize(y_pred_flat, bins) - 1

    # Etiquetas para los colores específicos
    labels = ["Negro (0-0.3)", "Gris (0.3-0.7)", "Blanco (0.7-1.0)"]
    cm = confusion_matrix(y_true_binned, y_pred_binned, labels=[0, 1, 2])

    # Mostrar la matriz de confusión con etiquetas de colores
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap='Blues', values_format='d')
    plt.title('Matriz de Confusión (Colores)')
    plt.show()

# Llamado de las funciones de visualización
print("Visualización de métricas y resultados...")
plot_training_curves(history)

# Realizar predicción en el conjunto de validación
y_pred_val = model.predict(X_val)

# Mostrar matriz de confusión basada en la predicción
plot_confusion_matrix(y_val, y_pred_val)

```Python

import matplotlib.pyplot as plt
import numpy as np

def plot_single_predicted_mask(mask_batch, image_index):
    """
    Grafica una máscara específica de un lote dado su índice.
    
    Parámetros:
    - mask_batch (np.array): Lote de máscaras predichas por el modelo, 
      con forma (n_imágenes, alto, ancho, canales).
    - image_index (int): Índice de la máscara que se desea graficar.
    
    Consideraciones:
    - Se asume que las máscaras están en escala de grises (1 canal).
    """
    
    # Validar que el índice esté dentro del rango del lote de máscaras
    if image_index >= mask_batch.shape[0] or image_index < 0:
        print(f"Índice {image_index} fuera de rango. El rango válido es de 0 a {mask_batch.shape[0]-1}.")
        return
    
    # Seleccionar la máscara correspondiente al índice
    mask = mask_batch[image_index, :, :, 0]  # Se selecciona el único canal de la máscara

    # Configurar y mostrar el gráfico
    plt.figure(figsize=(6, 6))
    plt.imshow(mask, cmap='gray')  # Mostrar en escala de grises
    plt.axis('off')  # Ocultar los ejes para una visualización más limpia
    plt.title(f'Máscara en el índice {image_index}')
    plt.show()

```Python

import numpy as np

def detect_centers_by_closest_pixels(mask_batch, num_pixels=314):
    """
    Detecta los centros de la fóvea y del disco óptico en un lote de máscaras predichas,
    basándose en la proximidad de los píxeles a 0 (fóvea) y a 1 (disco óptico).

    Parámetros:
    - mask_batch (np.array): Lote de máscaras predichas con forma (N, 224, 224, 1).
    - num_pixels (int): Número de píxeles a considerar para el cálculo del centroide (por defecto, 314).

    Retorna:
    - np.array: Arreglo de coordenadas con forma (N, 2, 2). Para cada imagen se devuelve:
                [centro_disco_óptico, centro_fóvea].
    """
    
    centers = []
    
    for mask in mask_batch:
        # Asegurar que la máscara tiene una sola dimensión (224, 224)
        mask = mask.squeeze()  # Elimina la dimensión extra si es (224, 224, 1)
        
        # Aplanar la imagen para facilitar la selección de píxeles
        flattened_mask = mask.flatten()
        
        # Calcular la distancia de cada píxel al valor de la fóvea (0) y al del disco óptico (1)
        distances_to_0 = np.abs(flattened_mask - 0)  # Distancia a la fóvea
        distances_to_1 = np.abs(flattened_mask - 1)  # Distancia al disco óptico
        
        # Seleccionar los índices de los 314 píxeles más cercanos a 0 (fóvea)
        fovea_indices = np.argsort(distances_to_0)[:num_pixels]
        
        # Seleccionar los índices de los 314 píxeles más cercanos a 1 (disco óptico)
        od_indices = np.argsort(distances_to_1)[:num_pixels]
        
        # Convertir los índices a coordenadas bidimensionales (fila, columna)
        fovea_coordinates = np.unravel_index(fovea_indices, mask.shape)
        od_coordinates = np.unravel_index(od_indices, mask.shape)
        
        # Calcular el centroide de las coordenadas más cercanas a la fóvea (si hay valores disponibles)
        fovea_center = np.mean(fovea_coordinates, axis=1) if len(fovea_coordinates[0]) > 0 else [None, None]
        
        # Calcular el centroide de las coordenadas más cercanas al disco óptico
        od_center = np.mean(od_coordinates, axis=1) if len(od_coordinates[0]) > 0 else [None, None]
        
        # Guardar los resultados
        centers.append([od_center, fovea_center])
    
    return np.array(centers)

```Python

Pred_coordinates = detect_centers_by_closest_pixels(Predict_mask)

```Python

import matplotlib.pyplot as plt

def plot_mask_and_centers(mask_batch, centers_batch, index):
    """
    Grafica una máscara predicha junto con los centros del disco óptico y de la fóvea.

    Parámetros:
    - mask_batch (np.array): Lote de máscaras predichas con forma (N, 224, 224, 1).
    - centers_batch (np.array): Lote de coordenadas de centros con forma (N, 2, 2), 
                                donde cada imagen tiene el centro del disco óptico y de la fóvea.
    - index (int): Índice de la imagen que se desea graficar.

    Retorno:
    - No retorna valores. Muestra una gráfica de la máscara con los centros marcados.
    """
    
    # Obtener la máscara correspondiente al índice dado
    mask = mask_batch[index].squeeze()  # Eliminar la dimensión adicional (224, 224)
    
    # Obtener las coordenadas de los centros del disco óptico y la fóvea
    od_center = centers_batch[index, 0]  # Centro del disco óptico
    fovea_center = centers_batch[index, 1]  # Centro de la fóvea
    
    # Graficar la máscara
    plt.imshow(mask, cmap='gray')
    
    # Graficar los centros en la máscara
    plt.scatter(od_center[1], od_center[0], color='green', label='Centro del Disco Óptico', 
                s=100, edgecolor='black')  # Marcador verde para el disco óptico
    plt.scatter(fovea_center[1], fovea_center[0], color='red', label='Centro de la Fóvea', 
                s=100, edgecolor='black')  # Marcador rojo para la fóvea
    
    # Mostrar leyenda y título
    plt.legend()
    plt.title(f'Máscara y Centros para la Imagen {index}')
    
    # Mostrar la gráfica
    plt.show()

# Carga del modelo y presentación de resultados:

```Python

# Librerías

import os
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

```Python

# Directorio de imágenes y CSVs
image_dir = "./Rets"
fovea_csv_path = "./IDRiD_Fovea_Center.csv"
od_csv_path = "./IDRiD_OD_Center.csv"
output_mask_dir = "./Masks"
os.makedirs(output_mask_dir, exist_ok=True)

```Python

fovea_data = pd.read_csv(fovea_csv_path)
od_data = pd.read_csv(od_csv_path)

```Python

# Combinar las coordenadas en un DataFrame
coordinates_df = pd.merge(fovea_data, od_data, on="Image No", suffixes=("_fovea", "_od"))

```Python

# Cargar imágenes y máscaras
image_filenames = sorted([f for f in os.listdir(image_dir) if f.endswith(".jpg")])
mask_filenames = sorted([f for f in os.listdir(output_mask_dir) if f.endswith(".png")])

X = []
y = []

```Python

print("Cargando imágenes y máscaras...")
for img_file, mask_file in tqdm(zip(image_filenames, mask_filenames), total=len(image_filenames)):
    img_path = os.path.join(image_dir, img_file)
    mask_path = os.path.join(output_mask_dir, mask_file)

    # Leer imagen y máscara
    img = cv2.imread(img_path)
    img = cv2.resize(img, image_size)  # Redimensionar para que quepa en memoria
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, image_size)
    mask = np.expand_dims(mask, axis=-1) / 255.0  # Normalizar la máscara

    X.append(img)
    y.append(mask)

X = np.array(X)
y = np.array(y)

```Python

from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError  # O usa la métrica exacta que usaste

# Cargar el modelo con la métrica personalizada registrada
modelo_cargado = load_model('attention_unet_bestv2.h5')#, custom_objects={'mse': MeanSquaredError()})

# Verificar la arquitectura del modelo cargado
modelo_cargado.summary()

````Python

# División en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

```Python

import numpy as np

# Buscar los índices
indices = [np.where(np.all(X == img, axis=(1, 2, 3)))[0][0] for img in X_val]

print("Índices del sub-conjunto en el arreglo principal:", indices)

```Python

#Coordenadas del conjunto de validación.

Coordinates_val = coordinates_df.loc[indices].reset_index(drop=True)
Coordinates_val

Predict_Mask = modelo_cargado.predict(X_val)

```Python 

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc

```Python

def plot_confusion_matrix(y_true, y_pred):
    """
    Grafica una matriz de confusión para evaluar la correspondencia entre las máscaras verdaderas y las predichas,
    clasificando los valores en tres categorías: negro, gris y blanco.

    Parámetros:
    - y_true (np.array): Máscara de valores verdaderos (ground truth).
    - y_pred (np.array): Máscara de valores predichos por el modelo.

    Funcionalidad:
    - Los valores de las máscaras se redondean y agrupan en tres categorías:
        - Negro: valores entre 0 y 0.3
        - Gris: valores entre 0.3 y 0.7
        - Blanco: valores entre 0.7 y 1.0
    - Calcula una matriz de confusión que muestra la relación entre las categorías predichas y las verdaderas.
    - Visualiza la matriz de confusión con colores para facilitar la interpretación.
    
    Retorno:
    - No retorna valores. Muestra una gráfica de la matriz de confusión.
    """
    
    # Aplanar las máscaras para facilitar el procesamiento
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()

    # Definir los límites para las categorías: negro, gris y blanco
    bins = [0, 0.3, 0.7, 1.01]  # Extiende ligeramente el último límite para incluir el 1.0
    y_true_binned = np.digitize(y_true_flat, bins) - 1  # Asignar cada valor a una categoría
    y_pred_binned = np.digitize(y_pred_flat, bins) - 1

    # Mapear las categorías a etiquetas específicas
    labels = ["Negro (0-0.3)", "Gris (0.3-0.7)", "Blanco (0.7-1.0)"]

    # Calcular la matriz de confusión
    cm = confusion_matrix(y_true_binned, y_pred_binned, labels=[0, 1, 2])

    # Mostrar la matriz de confusión
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap='Blues', values_format='d')  # Colores azules para la visualización
    plt.title('Matriz de Confusión (Colores)')
    plt.show()

```Python

import numpy as np

def detect_centers_by_closest_pixels(mask_batch, num_pixels=314):
    """
    Detectar los centros de la fóvea y del disco óptico en un lote de imágenes de máscaras predichas,
    basándose en la proximidad de los píxeles a 0 y 1.

    Parameters:
        mask_batch (np.array): Lote de máscaras predichas con forma (N, 224, 224, 1).
        num_pixels (int): Número de píxeles a considerar para el cálculo del centroide (314 en este caso).

    Returns:
        np.array: Arreglo de coordenadas, con forma (N, 2, 2), donde para cada imagen se devuelve 
                  el centro de la fóvea y el centro del disco óptico.
    """
    centers = []
    
    for mask in mask_batch:
        # Asegurarnos de que la máscara está en escala de grises (un solo canal)
        mask = mask.squeeze()  # Eliminar la dimensión adicional (224, 224)
        
        # Aplanar la imagen para facilitar la selección de píxeles
        flattened_mask = mask.flatten()
        
        # Calcular la distancia de cada píxel a 0 (fóvea) y a 1 (disco óptico)
        distances_to_0 = np.abs(flattened_mask - 0)  # Distancia a 0 (fóvea)
        distances_to_1 = np.abs(flattened_mask - 1)  # Distancia a 1 (disco óptico)
        
        # Obtener los índices de los 314 píxeles más cercanos a 0
        fovea_indices = np.argsort(distances_to_0)[:num_pixels]
        # Obtener los índices de los 314 píxeles más cercanos a 1
        od_indices = np.argsort(distances_to_1)[:num_pixels]
        
        # Convertir los índices a coordenadas (fila, columna)
        fovea_coordinates = np.unravel_index(fovea_indices, mask.shape)
        od_coordinates = np.unravel_index(od_indices, mask.shape)
        
        # Calcular el centroide de las coordenadas de los píxeles más cercanos a 0 (fóvea)
        fovea_center = np.mean(fovea_coordinates, axis=1) if len(fovea_coordinates[0]) > 0 else [None, None]
        
        # Calcular el centroide de las coordenadas de los píxeles más cercanos a 1 (disco óptico)
        od_center = np.mean(od_coordinates, axis=1) if len(od_coordinates[0]) > 0 else [None, None]
        
        # Guardar los resultados
        centers.append([od_center, fovea_center])
    
    return np.array(centers)

```Python

Pred_coordinates = detect_centers_by_closest_pixels(Predict_Mask)

```Python

import matplotlib.pyplot as plt

def plot_mask_and_centers(mask_batch, centers_batch, index):
    """
    Grafica la máscara predicha correspondiente al índice dado, junto con los puntos rojos
    que representan los centros del disco óptico y de la fóvea.

    Parameters:
        mask_batch (np.array): Lote de máscaras predichas de forma (N, 224, 224, 1).
        centers_batch (np.array): Lote de coordenadas de centros de forma (N, 2, 2),
                                  donde para cada imagen se devuelve el centro del disco
                                  óptico y de la fóvea.
        index (int): Índice de la imagen que se desea graficar.
    """
    # Obtener la máscara correspondiente al índice
    mask = mask_batch[index].squeeze()  # Eliminar la dimensión adicional (224, 224)
    
    # Obtener las coordenadas de los centros del disco óptico y la fóvea
    od_center = centers_batch[index, 0]  # Centro del disco óptico
    fovea_center = centers_batch[index, 1]  # Centro de la fóvea
    
    # Graficar la máscara
    plt.imshow(mask, cmap='gray')
    
    # Graficar los centros
    plt.scatter(od_center[1], od_center[0], color='green', label='Centro del Disco Óptico', s=100, edgecolor='black')
    plt.scatter(fovea_center[1], fovea_center[0], color='red', label='Centro de la Fóvea', s=100, edgecolor='black')
    
    # Mostrar leyenda y título
    plt.legend()
    plt.title(f'Máscara y Centros para la Imagen {index}')
    
    # Mostrar la gráfica
    plt.show()

```Python

def Resize(array, old_size, new_size):
    """
    Ajusta las coordenadas de un conjunto de puntos para un nuevo tamaño de imagen,
    escalando las posiciones y reordenando las coordenadas de (y, x) a (x, y).

    Parámetros:
    - array (np.array): Arreglo de puntos con forma (N, M, 2), donde cada punto 
                        tiene coordenadas (y, x).
    - old_size (tuple): Tamaño anterior de la imagen (alto, ancho).
    - new_size (tuple): Tamaño nuevo de la imagen (alto, ancho).

    Retorno:
    - np.array: Arreglo de puntos ajustados a la nueva escala, con coordenadas (x, y).

    Descripción de la funcionalidad:
    1. Reordena las coordenadas de cada punto para que sean (x, y) en lugar de (y, x).
    2. Escala las coordenadas según la proporción entre `new_size` y `old_size`.
    """

    # Reordenar las coordenadas de (y, x) a (x, y)
    New_order = np.array([array[i][:, [1, 0]] for i in range(array.shape[0])])

    # Escalar las coordenadas según la relación de tamaño nuevo y antiguo
    New_size = np.array([
        New_order[i] * [new_size[0] / old_size[0], new_size[1] / old_size[1]] 
        for i in range(New_order.shape[0])
    ])
    
    return New_size

```Python

# Redimensionar y ajustar las coordenadas predichas al tamaño original de la imagen
Pred_coordinates_new_size = Resize(Pred_coordinates, (224, 224), (4288, 2848))

# Crear un DataFrame con las coordenadas ajustadas de la fóvea y el disco óptico
Pred_coordinates_df = pd.DataFrame({
    "X-Coordinate_fovea_pred": Pred_coordinates_new_size[:, 1, 0],  # Coordenadas X de la fóvea
    "Y-Coordinate_fovea_pred": Pred_coordinates_new_size[:, 1, 1],  # Coordenadas Y de la fóvea
    "X-Coordinate_od_pred": Pred_coordinates_new_size[:, 0, 0],     # Coordenadas X del disco óptico
    "Y-Coordinate_od_pred": Pred_coordinates_new_size[:, 0, 1]      # Coordenadas Y del disco óptico
})

# Combinar las coordenadas verdaderas y predichas en un DataFrame intercalado
combined_DF = pd.DataFrame({
    'Image': Coordinates_val.iloc[:, 0],         # Nombres o identificadores de las imágenes
    'X_fovea': Coordinates_val.iloc[:, 1],       # Coordenada X verdadera de la fóvea
    'X_fovea_pred': Pred_coordinates_df.iloc[:, 0],  # Coordenada X predicha de la fóvea
    'Y_fovea': Coordinates_val.iloc[:, 2],       # Coordenada Y verdadera de la fóvea
    'Y_fovea_pred': Pred_coordinates_df.iloc[:, 1],  # Coordenada Y predicha de la fóvea
    'X_od': Coordinates_val.iloc[:, 3],          # Coordenada X verdadera del disco óptico
    'X_od_pred': Pred_coordinates_df.iloc[:, 2], # Coordenada X predicha del disco óptico
    'Y_od': Coordinates_val.iloc[:, 4],          # Coordenada Y verdadera del disco óptico
    'Y_od_pred': Pred_coordinates_df.iloc[:, 3], # Coordenada Y predicha del disco óptico
})

```Python

#Ejecución del código anterior:

x_real = np.array(combined_DF["X_od"])
x_pred = np.array(combined_DF["X_od_pred"])
y_real = np.array(combined_DF["Y_od"])
y_pred = np.array(combined_DF["Y_od_pred"])

RMSE_od, MAE_od = errors(x_real,y_real,x_pred,x_pred)

print('RMSE_od: {}\nMAE_od: {}'.format(RMSE_od,MAE_od))

```Python

def errors(x_real, y_real, x_pred, y_pred):
    """
    Calcula dos tipos diferentes de errores para evaluar el rendimiento del modelo.
    
    Actualmente implementa:
    1. **Raiz del error cuadrático medio (RMSE)**: mide la diferencia promedio al cuadrado entre los valores reales y predichos.
    2. **Error absoluto promedio (MAE)**: mide la diferencia promedio absoluta entre los valores reales y predichos.
    
    Parameters:
        x_real (array-like): Coordenadas X reales.
        y_real (array-like): Coordenadas Y reales.
        x_pred (array-like): Coordenadas X predichas.
        y_pred (array-like): Coordenadas Y predichas.

    Returns:
        tuple: Contiene el RMSE y el MAE.
    """
    
    # Cálculo del Error Cuadrático Medio (Root Mean Square Error)
    RMSE = (((x_real - x_pred)**2 + (y_real - y_pred)**2).sum() / 104) ** 0.5
    
    # Cálculo del Error Absoluto Promedio (Mean Absolute Error)
    MAE = (1 / 104) * (((x_real - x_pred)**2 + (y_real - y_pred)**2) ** 0.5).sum()

    return RMSE, MAE

```Python

import cv2
import matplotlib.pyplot as plt

def plot_image_with_points(image_path, target_size, points):
    """
    Grafica una imagen redimensionada con puntos ajustados correctamente al borde inferior izquierdo.

    Parameters:
    - image_path: ruta de la imagen
    - target_size: (ancho, alto) deseado para la imagen
    - points: lista de coordenadas (x, y) ya escaladas a la imagen original
    """
    # Cargar imagen RGB
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Redimensionar la imagen al tamaño deseado
    target_width, target_height = target_size
    resized_image = cv2.resize(image, (target_width, target_height))

    # Graficar la imagen
    plt.figure(figsize=(8, 8))
    plt.imshow(resized_image)

    # Colores y etiquetas para los puntos
    colors = ['red', 'blue', 'green', 'orange']
    labels = ['Real_Fovea_Center', 'Pred_Fovea_Center', 'Real_OD_Center', 'Pred_OD_Center']

    # Dibujar los puntos con las coordenadas tal cual están
    for i, (x, y) in enumerate(points):
        # No hacemos ningún ajuste o transformación a las coordenadas
        plt.scatter(x, y, c=colors[i], marker='x', s=100, label=labels[i])

    plt.legend()
    plt.title(f'IDRiD_513.jpg')
    plt.axis('off')
    plt.show()

```Python

# Ejemplo de uso
image_path = "Rets/IDRiD_513.jpg"
target_size = (400, 266)  # Dimensiones fijas de la imagen
points = [(combined_DF.loc[102][1]*400/4288, combined_DF.loc[102][3]*266/2848),
         (combined_DF.loc[102][2]*400/4288, combined_DF.loc[102][4]*266/2848),
         (combined_DF.loc[102][5]*400/4288, combined_DF.loc[102][7]*266/2848),
         (combined_DF.loc[102][6]*400/4288, combined_DF.loc[102][8]*266/2848)]  # Coordenadas originales

plot_image_with_points(image_path, target_size, points)

# **Segmentación**

# Librerías y funciones generales:

```python
#Librerías a importar:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split

```Python

# --- CARGA Y PREPROCESAMIENTO DE DATOS ---
def load_images_and_masks(image_dir, mask_dir, img_size, suffix):
    """
    Carga imágenes y máscaras desde directorios específicos, ajustando su tamaño y normalizando los datos.

    Parámetros:
    - image_dir (str): Ruta al directorio que contiene las imágenes.
    - mask_dir (str): Ruta al directorio que contiene las máscaras.
    - img_size (tuple): Dimensión objetivo para redimensionar las imágenes y máscaras (ancho, alto).
    - suffix (str): Sufijo que se añade al nombre del archivo de las máscaras.

    Retorna:
    - np.array: Arreglo de imágenes preprocesadas.
    - np.array: Arreglo de máscaras binarizadas.
    """
    # Construcción de las rutas para las imágenes y las máscaras
    # Las imágenes tienen nombres del tipo "IDRiD_01.jpg" hasta "IDRiD_81.jpg"
    image_paths = [os.path.join(image_dir, f"IDRiD_{str(i).zfill(2)}.jpg") for i in range(1, 82)]
    # Las máscaras tienen nombres del tipo "IDRiD_01_suffix.tif"
    mask_paths = [os.path.join(mask_dir, f"IDRiD_{str(i).zfill(2)}_{suffix}.tif") for i in range(1, 82)]

    images, masks = [], []  # Listas para almacenar las imágenes y las máscaras

    # Iteración conjunta sobre las rutas de las imágenes y las máscaras
    for img_path, mask_path in zip(image_paths, mask_paths):
        # Cargar y redimensionar la imagen
        img = load_img(img_path, target_size=img_size)
        # Cargar la máscara en modo escala de grises
        mask = load_img(mask_path, target_size=img_size, color_mode='grayscale')

        # Convertir la imagen a un arreglo NumPy y normalizar los valores a [0, 1]
        img = img_to_array(img) / 255.0
        # Convertir la máscara a un arreglo NumPy y normalizar
        mask = img_to_array(mask) / 255.0
        # Binarizar la máscara: valores mayores que 0 se convierten a 1
        mask = (mask > 0).astype(np.uint8)

        # Almacenar las imágenes y las máscaras en las listas
        images.append(img)
        masks.append(mask)

    # Convertir las listas a arreglos NumPy
    return np.array(images), np.array(masks)

```Python

# --- ARQUITECTURA DEL MODELO (Attention U-Net) ---
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Multiply, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model

# Bloque de atención
def attention_block(x, g, inter_channel):
    """
    Implementa un bloque de atención para enfocar la atención del modelo en características relevantes.

    Parámetros:
    - x: Tensor de entrada desde la rama de salto del codificador.
    - g: Tensor de entrada desde la rama de decodificación (up-sampled).
    - inter_channel: Número de canales intermedios.

    Retorna:
    - out: Tensor modulado por atención.
    """
    # Reducción de dimensiones del tensor x
    theta_x = Conv2D(inter_channel, (1, 1), strides=(1, 1), padding='same')(x)
    # Reducción de dimensiones del tensor g
    phi_g = Conv2D(inter_channel, (1, 1), strides=(1, 1), padding='same')(g)
    # Suma de características
    add = Add()([theta_x, phi_g])
    add = Activation('relu')(add)
    # Proyección para generar el mapa de atención
    psi = Conv2D(1, (1, 1), strides=(1, 1), padding='same')(add)
    psi = Activation('sigmoid')(psi)
    # Modulación del tensor original por el mapa de atención
    out = Multiply()([x, psi])
    return out

# Bloque de codificación
def encoder_block(input_tensor, num_filters):
    """
    Bloque de codificación con convoluciones, normalización y activación ReLU.

    Parámetros:
    - input_tensor: Tensor de entrada.
    - num_filters: Número de filtros para las capas convolucionales.

    Retorna:
    - x: Tensor procesado por las capas convolucionales.
    - p: Tensor tras la operación de max-pooling.
    """
    x = Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    # Reducir la dimensión espacial mediante max-pooling
    p = MaxPooling2D((2, 2))(x)
    return x, p

# Bloque de decodificación
def decoder_block(input_tensor, skip_features, num_filters):
    """
    Bloque de decodificación que combina características del codificador con las del decodificador.

    Parámetros:
    - input_tensor: Tensor de entrada desde el nivel anterior del decodificador.
    - skip_features: Características de la rama de salto del codificador.
    - num_filters: Número de filtros para las capas convolucionales.

    Retorna:
    - x: Tensor procesado por las capas convolucionales.
    """
    # Aumentar la dimensión espacial
    up = UpSampling2D((2, 2))(input_tensor)
    # Aplicar atención sobre las características del codificador
    attention = attention_block(skip_features, up, num_filters // 2)
    # Concatenación de las características del codificador y decodificador
    concat = concatenate([up, attention])
    x = Conv2D(num_filters, (3, 3), padding='same')(concat)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

# Construcción del modelo Attention U-Net
def attention_unet(input_shape, output_channels):
    """
    Construcción del modelo Attention U-Net.

    Parámetros:
    - input_shape: Dimensiones de la entrada (alto, ancho, canales).
    - output_channels: Número de canales de salida.

    Retorna:
    - model: Modelo Attention U-Net.
    """
    inputs = Input(input_shape)

    # Codificador
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    # Bottleneck
    b1 = Conv2D(1024, (3, 3), padding='same')(p4)
    b1 = BatchNormalization()(b1)
    b1 = Activation('relu')(b1)
    b1 = Conv2D(1024, (3, 3), padding='same')(b1)
    b1 = BatchNormalization()(b1)
    b1 = Activation('relu')(b1)

    # Decodificador
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    # Salida
    outputs = Conv2D(output_channels, (1, 1), activation='sigmoid')(d4)  # Activación sigmoide para segmentación binaria

    # Modelo
    model = Model(inputs, outputs, name="Attention_U-Net")
    return model

# Crear el modelo
input_shape = (320, 320, 3)   # Dimensiones de la entrada
output_channels = 1           # Número de canales de salida (1 para segmentación binaria)
model = attention_unet(input_shape, output_channels)

# Mostrar el resumen del modelo
model.summary()

```Python

import tensorflow.keras.backend as K

# Función de pérdida ponderada
def weighted_loss(y_true, y_pred):
    """
    Calcula una función de pérdida binaria ponderada para manejar el desbalance de clases.
    Esta pérdida es útil en problemas donde una clase es mucho más frecuente que la otra.

    Parámetros:
    -----------
    y_true : Tensor
        Valores verdaderos (etiquetas binarias).
    y_pred : Tensor
        Predicciones del modelo (valores continuos entre 0 y 1).
    
    Retorna:
    --------
    Tensor
        Valor de la pérdida ponderada promedio.
    """
    # Asegurar que y_true esté en formato flotante y convertirlo en un vector 1D
    y_true_f = K.cast(K.flatten(y_true), 'float32')
    y_pred_f = K.flatten(y_pred)

    # Pesos asignados a las clases (0.1 para la clase 0, 0.9 para la clase 1)
    # Estos valores deben ajustarse según el balance de las clases en tu problema
    class_weights = [0.1, 0.9]

    # Calcular el mapa de pesos: asignar pesos según las clases presentes
    # Si el valor es 1, se multiplica por class_weights[1]
    # Si el valor es 0, se multiplica por class_weights[0]
    weight_map = y_true_f * class_weights[1] + (1 - y_true_f) * class_weights[0]

    # Calcular la pérdida binaria entre las predicciones y las etiquetas reales
    loss = K.binary_crossentropy(y_true_f, y_pred_f)

    # Aplicar el mapa de pesos a la pérdida calculada
    weighted_loss = loss * weight_map

    # Retornar el promedio de la pérdida ponderada
    return K.mean(weighted_loss)

```Python

from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    """
    Generador de datos personalizado para entrenamiento de redes neuronales con Keras.
    Soporta la carga de imágenes y máscaras en lotes (batches) y permite aplicar 
    transformaciones de aumento de datos.

    Atributos:
    ----------
    - images : array-like
        Conjunto de imágenes de entrada.
    - masks : array-like
        Conjunto de máscaras correspondientes a las imágenes.
    - batch_size : int
        Tamaño del lote.
    - augment : bool
        Indica si se aplicarán transformaciones de aumento de datos.
    - image_datagen : ImageDataGenerator
        Generador para el aumento de imágenes.
    - mask_datagen : ImageDataGenerator
        Generador para el aumento de máscaras.

    Métodos:
    --------
    - __len__() : Devuelve el número total de lotes.
    - __getitem__(index) : Devuelve un lote de imágenes y máscaras en el índice dado.
    """
    
    def __init__(self, images, masks, batch_size, augment=False):
        """
        Inicializa el generador de datos.

        Parámetros:
        ----------
        - images : array-like
            Conjunto de imágenes de entrada.
        - masks : array-like
            Conjunto de máscaras correspondientes a las imágenes.
        - batch_size : int
            Tamaño del lote.
        - augment : bool, opcional (False por defecto)
            Si es True, se aplicarán transformaciones de aumento de datos.
        """
        self.images = images
        self.masks = masks
        self.batch_size = batch_size
        self.augment = augment

        # Configuración de aumento de datos para imágenes
        self.image_datagen = ImageDataGenerator(
            rotation_range=20,  # Rotación aleatoria en un rango de ±20 grados
            width_shift_range=0.1,  # Desplazamiento horizontal aleatorio
            height_shift_range=0.1,  # Desplazamiento vertical aleatorio
            shear_range=0.1,  # Transformación de corte aleatorio
            zoom_range=0.1,  # Zoom aleatorio
            horizontal_flip=True,  # Volteo horizontal aleatorio
            fill_mode='nearest'  # Modo de relleno para píxeles vacíos
        )

        # Configuración de aumento de datos para máscaras
        self.mask_datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            fill_mode='nearest'
        )

    def __len__(self):
        """
        Calcula el número de lotes por época.

        Retorna:
        --------
        int: Número total de lotes por época.
        """
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, index):
        """
        Obtiene un lote de imágenes y máscaras.

        Parámetros:
        -----------
        index : int
            Índice del lote.

        Retorna:
        --------
        tuple: Un lote de imágenes y sus máscaras correspondientes.
        """
        # Determinar el rango de datos para el lote actual
        start = index * self.batch_size
        end = (index + 1) * self.batch_size
        batch_images = self.images[start:end]
        batch_masks = self.masks[start:end]

        # Si se requiere aumento de datos
        if self.augment:
            # Generar una semilla aleatoria para sincronizar las transformaciones de imagen y máscara
            seed = np.random.randint(1e6)
            
            # Aplicar transformaciones de aumento a las imágenes
            batch_images = np.array([self.image_datagen.random_transform(img, seed=seed) for img in batch_images])
            
            # Aplicar las mismas transformaciones a las máscaras
            batch_masks = np.array([self.mask_datagen.random_transform(mask, seed=seed) for mask in batch_masks])

        return batch_images, batch_masks

```Python

from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt

class VisualizePredictionCallback(Callback):
    """
    Callback personalizado para visualizar las predicciones del modelo durante el entrenamiento.
    Al final de cada época, muestra la imagen original, la máscara verdadera y la predicción del modelo.

    Atributos:
    ----------
    - test_image: numpy array
        Imagen de prueba utilizada para generar predicciones.
    - test_mask: numpy array
        Máscara de prueba correspondiente a la imagen.
    """

    def __init__(self, test_image, test_mask):
        """
        Constructor de la clase.

        Parámetros:
        -----------
        - test_image: numpy array
            Imagen de prueba (debe estar preprocesada y lista para la entrada del modelo).
        - test_mask: numpy array
            Máscara correspondiente a la imagen de prueba.
        """
        super().__init__()
        self.test_image = test_image  # Imagen de prueba
        self.test_mask = test_mask    # Máscara de prueba

    def on_epoch_end(self, epoch, logs=None):
        """
        Método llamado al final de cada época. Genera la predicción del modelo
        y muestra la imagen, la máscara verdadera y la predicción.

        Parámetros:
        -----------
        - epoch: int
            Número de la época actual.
        - logs: dict, opcional
            Diccionario con métricas del modelo.
        """
        # Realizar la predicción con el modelo entrenado
        pred_mask = self.model.predict(self.test_image, verbose=0)
        
        # Extraer la máscara predicha (asumimos que el tensor tiene forma [1, alto, ancho, canales])
        pred_mask = pred_mask[0, :, :, 0]  # Extraer la primera muestra y el canal 0

        # Crear una figura con tres subgráficos
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

        # Mostrar la imagen original
        axes[0].imshow(self.test_image[0, :, :, 0], cmap="gray")
        axes[0].set_title("Imagen original")
        axes[0].axis("off")

        # Mostrar la máscara verdadera
        axes[1].imshow(self.test_mask[0, :, :, 0], cmap="gray")
        axes[1].set_title("Máscara verdadera")
        axes[1].axis("off")

        # Mostrar la máscara predicha
        axes[2].imshow(pred_mask, cmap="gray")
        axes[2].set_title(f"Predicción (Época {epoch + 1})")
        axes[2].axis("off")

        # Ajustar el diseño de la figura y mostrarla
        plt.tight_layout()
        plt.show()

```Python

from tensorflow.keras.callbacks import LearningRateScheduler

def poly_scheduler_OD(epoch, lr):
    """
    Calcula la tasa de aprendizaje utilizando la estrategia Poly, 
    que disminuye el learning rate de forma polinómica durante el entrenamiento.

    Parámetros:
    -----------
    - epoch: int
        La época actual del entrenamiento.
    - lr: float
        Tasa de aprendizaje actual (no se usa en el cálculo, pero es obligatorio por la API de Keras).

    Retorna:
    --------
    - new_lr: float
        Nueva tasa de aprendizaje calculada.
    """
    initial_learning_rate = 1e-2  # Tasa de aprendizaje inicial
    total_epochs = 80            # Número total de épocas para el entrenamiento
    power = 0.9                  # Parámetro que controla la velocidad de disminución

    # Fórmula de la estrategia Poly
    new_lr = initial_learning_rate * (1 - (epoch / total_epochs)) ** power
    return new_lr

# Crear el planificador de tasa de aprendizaje usando la estrategia Poly
poly_lr_scheduler_OD = LearningRateScheduler(poly_scheduler_OD)


class PrintLearningRateCallback_OD(tf.keras.callbacks.Callback):
    """
    Callback personalizado para imprimir la tasa de aprendizaje actual
    al inicio de cada época.
    """

    def on_epoch_begin(self, epoch, logs=None):
        """
        Método llamado al comienzo de cada época. Imprime la tasa de aprendizaje actual.

        Parámetros:
        -----------
        - epoch: int
            La época actual.
        - logs: dict, opcional
            Diccionario con información de la época (no utilizado aquí).
        """
        # Obtener la tasa de aprendizaje actual del optimizador
        lr = self.model.optimizer.learning_rate.numpy()
        print(f"Epoch {epoch + 1}: Learning rate is {lr:.6f}")


# Crear la instancia del callback para imprimir la tasa de aprendizaje
print_lr_callback_OD = PrintLearningRateCallback_OD()

```Python

from tensorflow.keras.callbacks import LearningRateScheduler

def poly_scheduler(epoch, lr):
    """
    Calcula la tasa de aprendizaje utilizando la estrategia Poly, 
    que disminuye la tasa de aprendizaje de forma polinómica a lo largo del entrenamiento.

    Parámetros:
    -----------
    - epoch: int
        La época actual durante el entrenamiento.
    - lr: float
        Tasa de aprendizaje actual (no se usa en esta implementación, pero es obligatorio por la API de Keras).

    Retorna:
    --------
    - new_lr: float
        Nueva tasa de aprendizaje calculada.
    """
    initial_learning_rate = 1e-2  # Tasa de aprendizaje inicial
    total_epochs = 180           # Número total de épocas para el entrenamiento
    power = 0.9                  # Parámetro que controla la velocidad de disminución

    # Fórmula de la estrategia Poly
    new_lr = initial_learning_rate * (1 - (epoch / total_epochs)) ** power
    return new_lr

# Crear el planificador de tasa de aprendizaje usando la estrategia Poly
poly_lr_scheduler = LearningRateScheduler(poly_scheduler)


class PrintLearningRateCallback(tf.keras.callbacks.Callback):
    """
    Callback personalizado para imprimir la tasa de aprendizaje actual
    al inicio de cada época durante el entrenamiento.
    """

    def on_epoch_begin(self, epoch, logs=None):
        """
        Método llamado al inicio de cada época para mostrar la tasa de aprendizaje.

        Parámetros:
        -----------
        - epoch: int
            La época actual.
        - logs: dict, opcional
            Información adicional sobre la época (no utilizado aquí).
        """
        # Obtener la tasa de aprendizaje actual del optimizador
        lr = self.model.optimizer.learning_rate.numpy()
        print(f"Epoch {epoch + 1}: Learning rate is {lr:.6f}")


# Instancia del callback para visualizar la tasa de aprendizaje
print_lr_callback = PrintLearningRateCallback()

```Python

from tensorflow.keras import backend as K

def dice_coefficient(y_true, y_pred):
    """
    Calcula el coeficiente de Dice, una métrica común para evaluar el rendimiento de modelos de segmentación.
    Esta métrica mide la similitud entre dos conjuntos, siendo 1 el valor ideal (superposición total) y 0 
    cuando no hay coincidencia.

    Parámetros:
    -----------
    - y_true: Tensor
        Máscara verdadera del conjunto de datos (ground truth).
    - y_pred: Tensor
        Máscara predicha por el modelo.

    Retorna:
    --------
    - float
        Valor del coeficiente de Dice.

    Nota:
    -----
    El coeficiente de Dice está definido como:
    Dice = (2 * |A ∩ B|) / (|A| + |B|)
    Donde A es la máscara verdadera y B es la máscara predicha.
    """
    # Factor suave para evitar división por cero
    smooth = 1e-6

    # Convertir `y_true` a tipo float32 para garantizar compatibilidad con `y_pred`
    y_true = K.cast(y_true, 'float32')

    # Calcular la intersección entre la máscara verdadera y la predicha
    intersection = K.sum(y_true * y_pred)

    # Calcular la unión de las máscaras
    union = K.sum(y_true) + K.sum(y_pred)

    # Retornar el coeficiente de Dice
    return (2. * intersection + smooth) / (union + smooth)

```Python

# --- Métricas de entrenamiento y validación ---
def plot_training_metrics(history):
    """
    Grafica las métricas de entrenamiento y validación almacenadas en `history`.
    
    Parámetros:
    -----------
    - history: History (objeto de Keras)
      Contiene el historial de entrenamiento del modelo, con métricas por época.
    """
    epochs = range(1, len(history.history['accuracy']) + 1)

    # Gráfico de precisión (accuracy)
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.plot(epochs, history.history['accuracy'], label='Precisión Entrenamiento')
    plt.plot(epochs, history.history['val_accuracy'], label='Precisión Validación')
    plt.title('Precisión de Entrenamiento y Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.legend()

    # Gráfico del coeficiente Dice
    plt.subplot(1, 3, 2)
    plt.plot(epochs, history.history['dice_coefficient'], label='Coeficiente Dice Entrenamiento')
    plt.plot(epochs, history.history['val_dice_coefficient'], label='Coeficiente Dice Validación')
    plt.title('Coeficiente Dice de Entrenamiento y Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Coeficiente Dice')
    plt.legend()

    # Gráfico de pérdida (loss)
    plt.subplot(1, 3, 3)
    plt.plot(epochs, history.history['loss'], label='Pérdida Entrenamiento')
    plt.plot(epochs, history.history['val_loss'], label='Pérdida Validación')
    plt.title('Pérdida de Entrenamiento y Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()

    plt.tight_layout()
    plt.show()

# --- Curva ROC ---
def plot_roc_curve(y_test, y_pred):
    """
    Grafica la curva ROC (Receiver Operating Characteristic) y calcula el AUC.

    Parámetros:
    -----------
    - y_test: Tensor o arreglo
      Valores verdaderos de las clases.
    - y_pred: Tensor o arreglo
      Valores predichos por el modelo.
    """
    # Calcular tasa de falsos positivos y verdaderos positivos
    fpr, tpr, _ = roc_curve(y_test.ravel(), y_pred.ravel())
    roc_auc = auc(fpr, tpr)

    # Gráfico de la curva ROC
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (Área = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Tasa de Falsos Positivos')
    plt.ylabel('Tasa de Verdaderos Positivos')
    plt.title('Curva ROC')
    plt.legend(loc='lower right')
    plt.show()

# --- Matriz de Confusión ---
def plot_confusion_matrix(y_true, y_pred):
    """
    Grafica la matriz de confusión.

    Parámetros:
    -----------
    - y_true: Tensor o arreglo
      Valores verdaderos de las clases.
    - y_pred: Tensor o arreglo
      Valores predichos por el modelo.
    """
    # Umbral para convertir predicciones a valores binarios (0 o 1)
    y_pred_thresholded = (y_pred > 0.5).astype(np.uint8)

    # Calcular la matriz de confusión
    cm = confusion_matrix(y_true.ravel(), y_pred_thresholded.ravel())

    # Gráfico de la matriz de confusión
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Matriz de Confusión')
    plt.xlabel('Predicción')
    plt.ylabel('Valor Verdadero')
    plt.show()

# Entrenamiento:

## Disco Óptico:

```Python

# --- PARÁMETROS DEL MODELO --- 

# Tamaño de las imágenes de entrada
IMG_SIZE = (320, 320)
# Especifica las dimensiones de las imágenes de entrada en formato (ancho, alto).
# Estas dimensiones serán utilizadas durante el preprocesamiento y el entrenamiento del modelo.

# Ruta del directorio que contiene las imágenes
img_path = "Rets"
# Carpeta que almacena las imágenes de entrada para el entrenamiento o validación.

# Ruta del directorio que contiene las máscaras correspondientes
mask_path = "Masks_OD"
# Carpeta que contiene las máscaras de segmentación asociadas a las imágenes de entrada.
# Se asume que las máscaras están alineadas pixel a pixel con las imágenes.

# Tamaño del lote (batch size)
BATCH_SIZE = 4
# Cantidad de imágenes y máscaras que se procesarán simultáneamente en cada iteración del entrenamiento.
# Un tamaño de lote pequeño (como 4) es útil si se trabaja con hardware de recursos limitados.

# Número total de épocas de entrenamiento
EPOCHS = 80
# Define cuántas veces el conjunto completo de datos pasará por el modelo durante el entrenamiento.
# Un valor de 80 suele ser suficiente para muchos problemas de segmentación, pero podría ajustarse según el desempeño.

```Python

# Carga las imágenes y sus respectivas máscaras para entrenamiento o evaluación.
images, masks = load_images_and_masks(img_path, mask_path, IMG_SIZE, 'OD')

```Python

# --- División del conjunto de datos ---
from sklearn.model_selection import train_test_split

# Primera división: conjunto de entrenamiento y temporal (30% para validación y prueba)
x_train, x_temp, y_train, y_temp = train_test_split(images, masks, test_size=0.3, random_state=42)

# Segunda división: conjunto de validación y conjunto de prueba (50% de x_temp para validación y 50% para prueba)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

```Python

# --- AUMENTO DE DATOS (DATA AUGMENTATION) ---
data_gen_args = dict(
    rotation_range=20,           # Rotación aleatoria de la imagen hasta 20 grados
    width_shift_range=0.1,       # Desplazamiento horizontal aleatorio de hasta el 10% del ancho de la imagen
    height_shift_range=0.1,      # Desplazamiento vertical aleatorio de hasta el 10% del alto de la imagen
    shear_range=0.1,             # Transformación de corte (shear) aleatoria de hasta el 10%
    zoom_range=0.1,              # Zoom aleatorio de hasta el 10%
    horizontal_flip=True,        # Volteo horizontal aleatorio
    fill_mode='nearest'          # Estrategia de relleno para píxeles vacíos resultantes de las transformaciones
)

# Creación de generadores de datos para imágenes y máscaras
image_datagen = ImageDataGenerator(**data_gen_args)  # Generador para imágenes con los argumentos definidos
mask_datagen = ImageDataGenerator(**data_gen_args)   # Generador para máscaras con las mismas transformaciones

```Python

# Ajuste de los generadores para normalización (si es necesario)
image_datagen.fit(x_train, augment=True)  # Ajusta el generador de imágenes a los datos de entrenamiento con aumento de datos
mask_datagen.fit(y_train, augment=True)   # Ajusta el generador de máscaras a los datos de entrenamiento con aumento de datos

```Python

# Crear generadores para el entrenamiento del modelo

# Fijamos una semilla para garantizar la reproducibilidad de los resultados
seed = 42

# Generador de imágenes
# image_datagen: es un objeto previamente configurado para aplicar transformaciones o aumentos de datos
# x_train: conjunto de imágenes de entrenamiento
# batch_size: tamaño del lote que se procesará en cada paso
# seed: semilla utilizada para sincronizar el generador de imágenes con el de máscaras
image_generator = image_datagen.flow(x_train, batch_size=BATCH_SIZE, seed=seed)

# Generador de máscaras
# mask_datagen: es un objeto similar a image_datagen pero diseñado para las máscaras de segmentación
# y_train: conjunto de máscaras de entrenamiento correspondiente a las imágenes en x_train
mask_generator = mask_datagen.flow(y_train, batch_size=BATCH_SIZE, seed=seed)

# Combinar generadores de imágenes y máscaras
# zip empareja cada lote de imágenes con su lote correspondiente de máscaras
train_generator = zip(image_generator, mask_generator)

```Python

# Crear el modelo de red neuronal para segmentación

# Dimensiones de la entrada del modelo (alto, ancho, canales de color)
# En este caso, imágenes de 320x320 píxeles con 3 canales de color (RGB)
input_shape = (320, 320, 3)   

# Número de canales de salida
# Para una segmentación binaria (fondo vs. objeto), se utiliza un canal de salida
output_channels = 1           

# Definición del modelo usando una arquitectura U-Net con bloques de atención
# attention_unet: función previamente definida o importada que crea el modelo basado en U-Net
model_OD = attention_unet(input_shape, output_channels)

# Mostrar el resumen del modelo
# summary() proporciona información sobre las capas, el número de parámetros entrenables
# y las dimensiones de salida de cada capa
model_OD.summary()

```Python

# Creación de generadores personalizados para el entrenamiento y validación del modelo

# Generador para los datos de entrenamiento
# DataGenerator: clase personalizada previamente definida para generar lotes de datos en tiempo real
# x_train: conjunto de imágenes de entrenamiento
# y_train: conjunto de máscaras de entrenamiento correspondientes
# BATCH_SIZE: tamaño del lote procesado en cada paso
# augment=True: habilita el aumento de datos (transformaciones aleatorias) para mejorar la generalización del modelo
train_generator = DataGenerator(x_train, y_train, BATCH_SIZE, augment=True)

# Generador para los datos de validación
# x_val: conjunto de imágenes de validación
# y_val: conjunto de máscaras de validación correspondientes
# augment=False: no se aplica aumento de datos durante la validación, ya que los datos deben mantenerse consistentes
val_generator = DataGenerator(x_val, y_val, BATCH_SIZE, augment=False)

```Python

# Seleccionar una imagen y su máscara correspondiente del conjunto de validación

# test_image contiene una imagen de validación
# Seleccionamos solo el primer elemento del conjunto x_val
# La dimensión resultante es (1, 320, 320, 3): un lote de una imagen RGB de 320x320 píxeles
test_image = x_val[0:1]  

# test_mask contiene la máscara correspondiente a test_image
# Seleccionamos el primer elemento de y_val, con una dimensión de (1, 320, 320, 1)
# La máscara es en escala de grises, generalmente binaria (0 para fondo y 1 para objeto)
test_mask = y_val[0:1]  

# Crear el callback para visualizar las predicciones durante el entrenamiento
# VisualizePredictionCallback: clase personalizada que permite observar las predicciones del modelo
# sobre test_image y compararlas con test_mask
visualize_callback = VisualizePredictionCallback(test_image, test_mask)

```Python

# Ajustar el modelo con el generador corregido

# Definir el modelo basado en la arquitectura Attention U-Net
# input_shape: dimensiones de la entrada de la red (320x320 píxeles, 3 canales para imágenes RGB)
# output_channels: número de canales de salida; 1 para segmentación binaria (fondo vs objeto)
input_shape = (320, 320, 3)  
output_channels = 1  
model_OD = attention_unet(input_shape, output_channels)

# Compilar el modelo
# optimizer='adam': algoritmo de optimización eficiente para el ajuste de pesos
# loss=weighted_loss: función de pérdida personalizada ponderada, ideal para manejar clases desbalanceadas
# metrics=['accuracy', dice_coefficient]: métricas para evaluar el desempeño; accuracy para clasificación
# y dice_coefficient para evaluar la calidad de la segmentación
model_OD.compile(optimizer='adam', loss=weighted_loss, metrics=['accuracy', dice_coefficient])

# Entrenar el modelo
history_OD = model_OD.fit(
    train_generator,  # generador de datos para el entrenamiento
    validation_data=(x_val, y_val),  # datos de validación: imágenes y máscaras correspondientes
    epochs=EPOCHS,  # número total de épocas para el entrenamiento
    steps_per_epoch=len(train_generator),  # número de pasos por cada época
    callbacks=[poly_lr_scheduler_OD, print_lr_callback_OD]  # callbacks personalizados para ajustar el aprendizaje
)

```Python

# --- EVALUACIÓN DEL MODELO ---
# Evaluar el modelo con datos de prueba
# x_test: conjunto de imágenes de prueba
# y_test: conjunto de máscaras correspondientes para la prueba
# La función evaluate devuelve una lista con el valor de la función de pérdida y las métricas definidas durante la compilación
eval_results = model_OD.evaluate(x_test, y_test)

# Mostrar los resultados de la evaluación
# eval_results[0]: pérdida (loss) en el conjunto de prueba
# eval_results[1]: precisión (accuracy) en el conjunto de prueba
print(f"Test Loss: {eval_results[0]}, Test Accuracy: {eval_results[1]}")

# Guardar el modelo entrenado en un archivo
# Se guarda en formato HDF5 (.h5), que permite almacenar tanto la arquitectura como los pesos del modelo
model_OD.save("unetplusplus_opticDisc_segmentation.h5")

```Python

# --- GENERAR GRÁFICAS Y EVALUACIONES VISUALES ---

# Gráfica de las métricas del entrenamiento
# plot_training_metrics: función personalizada para graficar el historial de entrenamiento
# Se pueden visualizar métricas como pérdida (loss), precisión (accuracy) y el coeficiente Dice
plot_training_metrics(history_OD)

# Generar predicciones para el conjunto de prueba
# model.predict(x_test): realiza predicciones para cada imagen en x_test
# y_pred contendrá las máscaras predichas por el modelo
y_pred = model_OD.predict(x_test)

# Graficar la curva ROC (Receiver Operating Characteristic)
# plot_roc_curve: función personalizada para graficar la curva ROC
# Esta curva evalúa el desempeño del modelo considerando la relación entre sensibilidad y especificidad
plot_roc_curve(y_test, y_pred)

# Graficar la matriz de confusión
# plot_confusion_matrix: función personalizada para mostrar la matriz de confusión
# Evalúa el número de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos
plot_confusion_matrix(y_test, y_pred)

A partir de este punto, no se comentará el resto, ya que sigue prácticamente el mismo enfoque aplicado en el disco óptico.

## Exudados Fuertes:

```Python

# --- PARÁMETROS ---
IMG_SIZE = (400, 400)
img_path = "Rets"
mask_path = "Masks_EX"
BATCH_SIZE = 4
EPOCHS = 180
LEARNING_RATE = 1e-4

```Python

images, masks = load_images_and_masks(img_path, mask_path, IMG_SIZE,'EX')

```Python

from sklearn.model_selection import train_test_split
x_train, x_temp, y_train, y_temp = train_test_split(images, masks, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

```Python

data_gen_args = dict(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

```Python

image_datagen.fit(x_train, augment=True)
mask_datagen.fit(y_train, augment=True)

```Python

seed = 42
image_generator = image_datagen.flow(x_train, batch_size=BATCH_SIZE, seed=seed)
mask_generator = mask_datagen.flow(y_train, batch_size=BATCH_SIZE, seed=seed)
train_generator = zip(image_generator, mask_generator)

```Python

# Crear el modelo
input_shape = (400, 400, 3)   # Dimensiones de la entrada
output_channels = 1           # Tipo de salida (ej. 1 para segmentación binaria)
model_EX = attention_unet(input_shape, output_channels)

# Resumen del modelo
model_EX.summary()

```Python

# Usar el nuevo generador
train_generator = DataGenerator(x_train, y_train, BATCH_SIZE, augment=True)
val_generator = DataGenerator(x_val, y_val, BATCH_SIZE, augment=False)

```Python

test_image = x_val[0:1]  
test_mask = y_val[0:1]  
# Crear el callback
visualize_callback = VisualizePredictionCallback(test_image, test_mask)

```Python

# Ajustar el modelo con el generador corregido

# Definir el modelo
input_shape = (400, 400, 3)  # Ejemplo de tamaño de imagen y 1 canal (escala de grises)
output_channels = 1  # Por ejemplo, para segmentación binaria
model_EX = attention_unet(input_shape, output_channels)

# Compilar el modelo
model_EX.compile(optimizer='adam', loss=weighted_loss, metrics=['accuracy', dice_coefficient])

history_EX = model_EX.fit(
    train_generator,
    validation_data=(x_val, y_val),
    epochs=EPOCHS,
    steps_per_epoch=len(train_generator),
    callbacks=[poly_lr_scheduler,print_lr_callback]
)

```Python 

eval_results = model_EX.evaluate(x_test, y_test)
print(f"Test Loss: {eval_results[0]}, Test Accuracy: {eval_results[1]}")

model_EX.save("unetplusplus_Exudates_segmentation.h5")

```Python

plot_training_metrics(history_EX)


y_pred = model.predict(x_test)


plot_roc_curve(y_test, y_pred)


plot_confusion_matrix(y_test, y_pred)

## Exudados Suaves:

```Python

IMG_SIZE = (400, 400)
img_path = "Rets"
mask_path = "Masks_SE"
BATCH_SIZE = 4
EPOCHS = 180
LEARNING_RATE = 1e-4

```Python

images, masks = load_images_and_masks(img_path, mask_path, IMG_SIZE,'SE')

```Python

from sklearn.model_selection import train_test_split
x_train, x_temp, y_train, y_temp = train_test_split(images, masks, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

```Python

data_gen_args = dict(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

```Python

image_datagen.fit(x_train, augment=True)
mask_datagen.fit(y_train, augment=True)

```Python

seed = 42
image_generator = image_datagen.flow(x_train, batch_size=BATCH_SIZE, seed=seed)
mask_generator = mask_datagen.flow(y_train, batch_size=BATCH_SIZE, seed=seed)
train_generator = zip(image_generator, mask_generator)

```Python

# Crear el modelo
input_shape = (400, 400, 3)   # Dimensiones de la entrada
output_channels = 1           # Tipo de salida (ej. 1 para segmentación binaria)
model_SE = attention_unet(input_shape, output_channels)

# Resumen del modelo
model_SE.summary()

```Python

# Usar el nuevo generador
train_generator = DataGenerator(x_train, y_train, BATCH_SIZE, augment=True)
val_generator = DataGenerator(x_val, y_val, BATCH_SIZE, augment=False)

```Python

test_image = x_val[0:1]  # Dimensión: (1, 320, 320, 3)
test_mask = y_val[0:1]  # Dimensión: (1, 320, 320, 1)
# Crear el callback
visualize_callback = VisualizePredictionCallback(test_image, test_mask)

```Python

# Ajustar el modelo con el generador corregido

# Definir el modelo
input_shape = (400, 400, 3)  # Ejemplo de tamaño de imagen y 1 canal (escala de grises)
output_channels = 1  # Por ejemplo, para segmentación binaria
model_SE = attention_unet(input_shape, output_channels)

# Compilar el modelo
model_SE.compile(optimizer='adam', loss=weighted_loss, metrics=['accuracy', dice_coefficient])

history_SE = model_SE.fit(
    train_generator,
    validation_data=(x_val, y_val),
    epochs=EPOCHS,
    steps_per_epoch=len(train_generator),
    callbacks=[poly_lr_scheduler,print_lr_callback]
)

```Python

eval_results = model_SE.evaluate(x_test, y_test)
print(f"Test Loss: {eval_results[0]}, Test Accuracy: {eval_results[1]}")


model_SE.save("unetplusplus_Soft_exudates_segmentation.h5")

```Python

plot_training_metrics(history_SE)


y_pred = model.predict(x_test)


plot_roc_curve(y_test, y_pred)


plot_confusion_matrix(y_test, y_pred)

## Microaneurismas:

```Python

IMG_SIZE = (400, 400)
img_path = "Rets"
mask_path = "Masks_MA"
BATCH_SIZE = 4
EPOCHS = 180
LEARNING_RATE = 1e-4

```Python

images, masks = load_images_and_masks(img_path, mask_path, IMG_SIZE,'MA')

```Python

from sklearn.model_selection import train_test_split
x_train, x_temp, y_train, y_temp = train_test_split(images, masks, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

```Python

data_gen_args = dict(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

```Python

image_datagen.fit(x_train, augment=True)
mask_datagen.fit(y_train, augment=True)

```Python

seed = 42
image_generator = image_datagen.flow(x_train, batch_size=BATCH_SIZE, seed=seed)
mask_generator = mask_datagen.flow(y_train, batch_size=BATCH_SIZE, seed=seed)
train_generator = zip(image_generator, mask_generator)

```Python

# Crear el modelo
input_shape = (400, 400, 3)   # Dimensiones de la entrada
output_channels = 1           # Tipo de salida (ej. 1 para segmentación binaria)
model_MA = attention_unet(input_shape, output_channels)

# Resumen del modelo
model_MA.summary()

```Python

# Usar el nuevo generador
train_generator = DataGenerator(x_train, y_train, BATCH_SIZE, augment=True)
val_generator = DataGenerator(x_val, y_val, BATCH_SIZE, augment=False)

```Python

test_image = x_val[0:1]  # Dimensión: (1, 320, 320, 3)
test_mask = y_val[0:1]  # Dimensión: (1, 320, 320, 1)
# Crear el callback
visualize_callback = VisualizePredictionCallback(test_image, test_mask)

```Python

# Ajustar el modelo con el generador corregido

# Definir el modelo
input_shape = (400, 400, 3)  # Ejemplo de tamaño de imagen y 1 canal (escala de grises)
output_channels = 1  # Por ejemplo, para segmentación binaria
model_MA = attention_unet(input_shape, output_channels)

# Compilar el modelo
model_MA.compile(optimizer='adam', loss=weighted_loss, metrics=['accuracy', dice_coefficient])

history_MA = model_MA.fit(
    train_generator,
    validation_data=(x_val, y_val),
    epochs=EPOCHS,
    steps_per_epoch=len(train_generator),
    callbacks=[poly_lr_scheduler,print_lr_callback]
)

```Python

eval_results = model_MA.evaluate(x_test, y_test)
print(f"Test Loss: {eval_results[0]}, Test Accuracy: {eval_results[1]}")

model_MA.save("unetplusplus_microaneurysms_segmentation.h5")

```Python

plot_training_metrics(history_MA)


y_pred = model.predict(x_test)


plot_roc_curve(y_test, y_pred)


plot_confusion_matrix(y_test, y_pred)

## Hemorragias:

```Python

IMG_SIZE = (400, 400)
img_path = "Rets"
mask_path = "Masks_HE"
BATCH_SIZE = 4
EPOCHS = 180
LEARNING_RATE = 1e-4

```Python

images, masks = load_images_and_masks(img_path, mask_path, IMG_SIZE,'HE')

```Python

from sklearn.model_selection import train_test_split
x_train, x_temp, y_train, y_temp = train_test_split(images, masks, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

```Python

data_gen_args = dict(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

```Python

image_datagen.fit(x_train, augment=True)
mask_datagen.fit(y_train, augment=True)

```Python

seed = 42
image_generator = image_datagen.flow(x_train, batch_size=BATCH_SIZE, seed=seed)
mask_generator = mask_datagen.flow(y_train, batch_size=BATCH_SIZE, seed=seed)
train_generator = zip(image_generator, mask_generator)

```Python

# Crear el modelo
input_shape = (400, 400, 3)   # Dimensiones de la entrada
output_channels = 1           # Tipo de salida (ej. 1 para segmentación binaria)
model_HE = attention_unet(input_shape, output_channels)

# Resumen del modelo
model_HE.summary()

```Python

# Usar el nuevo generador
train_generator = DataGenerator(x_train, y_train, BATCH_SIZE, augment=True)
val_generator = DataGenerator(x_val, y_val, BATCH_SIZE, augment=False)

```Python

test_image = x_val[0:1]  # Dimensión: (1, 320, 320, 3)
test_mask = y_val[0:1]  # Dimensión: (1, 320, 320, 1)
# Crear el callback
visualize_callback = VisualizePredictionCallback(test_image, test_mask)

```Python

# Ajustar el modelo con el generador corregido

# Definir el modelo
input_shape = (400, 400, 3)  # Ejemplo de tamaño de imagen y 1 canal (escala de grises)
output_channels = 1  # Por ejemplo, para segmentación binaria
model_HE = attention_unet(input_shape, output_channels)

# Compilar el modelo
model_HE.compile(optimizer='adam', loss=weighted_loss, metrics=['accuracy', dice_coefficient])

history_HE = model_HE.fit(
    train_generator,
    validation_data=(x_val, y_val),
    epochs=EPOCHS,
    steps_per_epoch=len(train_generator),
    callbacks=[poly_lr_scheduler,print_lr_callback]
)

```Python

eval_results = model_MA.evaluate(x_test, y_test)
print(f"Test Loss: {eval_results[0]}, Test Accuracy: {eval_results[1]}")

model_MA.save("unetplusplus_Hemorrhages_segmentation.h5")

```Python

plot_training_metrics(history_HE)


y_pred = model.predict(x_test)


plot_roc_curve(y_test, y_pred)


plot_confusion_matrix(y_test, y_pred)

# Guardar las historias:

```Python

import json

# Guardar history_OD como JSON
with open('history_OD.json', 'w') as file:
    json.dump(history_OD.history, file)

```Python

import json

# Guardar history_EX como JSON
with open('history_EX.json', 'w') as file:
    json.dump(history_EX.history, file)

```Python

import json

# Guardar history_MA como JSON
with open('history_MA.json', 'w') as file:
    json.dump(history_MA.history, file)

```Python

import json

# Guardar history_SE como JSON
with open('history_SE.json', 'w') as file:
    json.dump(history_SE.history, file)

```Python

import json

# Guardar history_HE como JSON
with open('history_HE.json', 'w') as file:
    json.dump(history_HE.history, file)

# Carga de los modelos:

## Disco óptico:

```Python

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

```Python

# Importar la función para cargar modelos preentrenados de Keras
from tensorflow.keras.models import load_model  

# Cargar el modelo previamente guardado
# 'unetplusplus_opticDisc_segmentation.h5': archivo que contiene el modelo entrenado
# compile=False: evita compilar el modelo al cargarlo, útil si solo se requiere para inferencia
modelo = load_model('unetplusplus_opticDisc_segmentation.h5', compile=False)

# Mostrar el resumen del modelo
# Proporciona información detallada sobre la arquitectura del modelo, el número de capas y parámetros
modelo.summary()

```Python

# --- PARÁMETROS ---
IMG_SIZE = (320, 320)
img_path = "Rets"
mask_path = "Masks_OD"
BATCH_SIZE = 4
EPOCHS = 50
LEARNING_RATE = 1e-4

```Python

# --- CARGA Y PREPROCESAMIENTO DE DATOS ---

def load_images_and_masks(image_dir, mask_dir, img_size):
    """
    Carga y preprocesa imágenes y sus máscaras correspondientes desde directorios específicos.

    Parámetros:
    - image_dir: ruta del directorio que contiene las imágenes.
    - mask_dir: ruta del directorio que contiene las máscaras.
    - img_size: tamaño objetivo al que se redimensionarán las imágenes y máscaras (alto, ancho).

    Retorno:
    - images: arreglo numpy con las imágenes preprocesadas.
    - masks: arreglo numpy con las máscaras binarizadas.
    """

    # Generar las rutas de las imágenes y sus máscaras
    # Los archivos de imágenes siguen el patrón "IDRiD_XX.jpg"
    # Las máscaras siguen el patrón "IDRiD_XX_OD.tif"
    image_paths = [os.path.join(image_dir, f"IDRiD_{str(i).zfill(2)}.jpg") for i in range(1, 82)]
    mask_paths = [os.path.join(mask_dir, f"IDRiD_{str(i).zfill(2)}_OD.tif") for i in range(1, 82)]

    images, masks = [], []  # Listas para almacenar imágenes y máscaras cargadas

    # Iterar sobre pares de imágenes y máscaras
    for img_path, mask_path in zip(image_paths, mask_paths):
        # Cargar y redimensionar la imagen
        img = load_img(img_path, target_size=img_size)

        # Cargar y redimensionar la máscara en escala de grises
        mask = load_img(mask_path, target_size=img_size, color_mode='grayscale')

        # Convertir la imagen y la máscara a arreglos numpy y normalizarlos en el rango [0, 1]
        img = img_to_array(img) / 255.0
        mask = img_to_array(mask) / 255.0

        # Binarizar la máscara: los valores mayores que 0 se convierten en 1 (segmentación binaria)
        mask = (mask > 0).astype(np.uint8)

        # Almacenar las imágenes y máscaras preprocesadas
        images.append(img)
        masks.append(mask)

    # Convertir listas a arreglos numpy
    return np.array(images), np.array(masks)

```Python

images, masks = load_images_and_masks(img_path, mask_path, IMG_SIZE)

```Python

# Partir dataset
from sklearn.model_selection import train_test_split
x_train, x_temp, y_train, y_temp = train_test_split(images, masks, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

```Python

pred_mask = modelo.predict(x_test)

```Python

import matplotlib.pyplot as plt

def mostrar_comparacion(imagenes_rgb, mascaras_reales, mascaras_predichas, indice):
    """
    Grafica una retinografía, su máscara real y su máscara predicha de manera contigua.

    Parámetros:
    - imagenes_rgb: numpy array de imágenes RGB con forma (n, 400, 400, 3)
      Conjunto de imágenes originales (retinografías).
    - mascaras_reales: numpy array de máscaras reales con forma (n, 400, 400, 1)
      Conjunto de máscaras reales asociadas a las imágenes.
    - mascaras_predichas: numpy array de máscaras predichas con forma (n, 400, 400, 1)
      Conjunto de máscaras generadas por el modelo.
    - indice: int, índice de la imagen a mostrar.
    """
    
    # Extraer la imagen y las máscaras correspondientes
    imagen = imagenes_rgb[indice]  # Imagen original RGB
    mascara_real = mascaras_reales[indice].squeeze()  # Máscara real ajustada para 2D
    mascara_predicha = mascaras_predichas[indice].squeeze()  # Máscara predicha ajustada para 2D

    # Configurar la figura para mostrar las tres imágenes
    plt.figure(figsize=(12, 4))

    # Mostrar la imagen original
    plt.subplot(1, 3, 1)
    plt.imshow(imagen)
    plt.title("Retinografía")
    plt.axis("off")  # Ocultar ejes

    # Mostrar la máscara real en escala de grises
    plt.subplot(1, 3, 2)
    plt.imshow(mascara_real, cmap='gray')
    plt.title("Máscara Real")
    plt.axis("off")

    # Mostrar la máscara predicha en escala de grises
    plt.subplot(1, 3, 3)
    plt.imshow(mascara_predicha, cmap='gray')
    plt.title("Máscara Predicha")
    plt.axis("off")

    # Ajustar el diseño de la figura y mostrarla
    plt.tight_layout()
    plt.show()

```Python 

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

# Obtener las puntuaciones del modelo para el conjunto de validación
# modelo.predict(x_val): genera las probabilidades predichas para cada pixel (en este caso, la clase positiva)
# flatten(): convierte el arreglo en un vector unidimensional
y_scores = modelo.predict(x_val).flatten()  

# Asegurarse de que las etiquetas de las máscaras también sean unidimensionales
y_val = y_val.flatten()  

# Calcular la curva Precision-Recall
# precision: valores de precisión a diferentes umbrales
# recall: valores de recuperación (recall) correspondientes
# _: umbrales utilizados para calcular precision y recall
precision, recall, _ = precision_recall_curve(y_val, y_scores)

# Graficar la curva Precision-Recall
plt.plot(recall, precision)
plt.title('Curva Precision-Recall')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

El procedimiento para cargar los otros modelos y generar sus respectivas gráficas es idéntico a este.